# Sentiment Analysis

This notebook classifies the tweets of each candidate in sentiment (POS/NEG/NEU) and emotion (joy/sadness/anger/surprise/disgust/fear/other). The classifier output is, for each category, the probability of belong to that category. BERT fine-tuned for Spanish tweets library (PySentimiento) is used.

- Input: tweets of candidates
- Output: probability of sentiments and emotions for each tweet

In [ ]:
!pip install pyathena "pymongo[srv]" pysentimiento

     |████████████████████████████████| 131 kB 6.5 MB/s 
     |████████████████████████████████| 8.0 MB 25.3 MB/s 
     |████████████████████████████████| 79 kB 6.5 MB/s 
     |████████████████████████████████| 138 kB 65.7 MB/s 
     |████████████████████████████████| 287 kB 68.6 MB/s 
     |████████████████████████████████| 170 kB 72.4 MB/s 
     |████████████████████████████████| 2.9 MB 34.5 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
     |████████████████████████████████| 125 kB 64.3 MB/s 
     |████████████████████████████████| 243 kB 55.9 MB/s 
     |████████████████████████████████| 1.3 MB 26.8 MB/s 
     |████████████████████████████████| 127 kB 49.4 MB/s 
     |████████████████████████████████| 271 kB 53.3 MB/s 
     |████████████████████████████████| 160 kB 53.8 MB/s 
     |████████████████████████████████| 188 kB 71.7 MB/s 
     |████████████████████████████████| 3.3 MB 31.9 MB/s 
     |████████████████████████████████| 895 kB 51.4 MB/s 
     |█████████████

In [ ]:
import logging
import os
import pandas as pd
import pymongo
from pyathena import connect
from pysentimiento import EmotionAnalyzer
import time
import tqdm

mongo_client = pymongo.MongoClient(os.environ["MONGODB_URL"])
twitter_db = mongo_client.TwitterConstituyenteDB
test_db = mongo_client.testdb

conn = connect(s3_staging_dir=os.environ["AWS_ATHENA_S3_STAGING_DIR"], region_name=os.environ["AWS_REGION"])

query = """
SELECT * FROM "twitter-constituyente"."constituyentes";
"""

df = pd.read_sql(query, conn)

candidates_ids = df["user__id_str"].dropna().to_list()

In [ ]:
c = twitter_db.tweets.count_documents({"datetime":{"$exists":False}})
c

0

In [ ]:
def bulk_write_to_mongo(collection, data):
    to_insert = len(data)
    try:
        if to_insert > 0:
            collection.insert_many(data, ordered=False)
        return to_insert, 0
    except BulkWriteError as e:
        log.error("BulkWriteError")
        inserted = e.details["nInserted"]
        return inserted, to_insert - inserted

In [ ]:
def create_output_sentiment_analysis(i, sentiment, statuses):
  out = sentiment[i].probas
  out["output"] = sentiment[i].output
  out["tweet__id_str"] = statuses[i]["id_str"]
  out["user__id_str"] = statuses[i]["user"]["id_str"]
  return out


In [ ]:
# twitter_db.emotion_analysis.create_index("tweet__id_str", unique=True)

'tweet__id_str_1'

In [ ]:
analyzer = EmotionAnalyzer(lang="es")

loading configuration file https://huggingface.co/finiteautomata/beto-emotion-analysis/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/b75b62ad64772a1df4c46943b8729e726a8bcc147e845197d591ebde2d1430b2.516dbdb8064a2498055e9af6e5d92ae5a4583bcefe0ca71535c71f42cf513138
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-cased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "others",
    "1": "joy",
    "2": "sadness",
    "3": "anger",
    "4": "surprise",
    "5": "disgust",
    "6": "fear"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "anger": 3,
    "disgust": 5,
    "fear": 6,
    "joy": 1,
    "others": 0,
    "sadness": 2,
    "surprise": 4
  },
  "layer_norm_eps": 1e-12,
  "ma

In [ ]:
downloaded_ids = list(twitter_db.emotion_analysis.distinct("tweet__id_str"))

In [ ]:
len(downloaded_ids)

0

In [ ]:
tweets_text = [status["full_text"] for status in t]
sentiment = analyzer.predict(tweets_text)
create_output_sentiment_analysis(0, sentiment, t)

  0%|          | 0/1 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 1
  Batch size = 32


1


{'anger': 0.0016183812404051423,
 'disgust': 0.000781764043495059,
 'fear': 0.0010986702982336283,
 'joy': 0.0037622980307787657,
 'others': 0.9809114933013916,
 'output': 'others',
 'sadness': 0.001357990549877286,
 'surprise': 0.010469348169863224,
 'tweet__id_str': '1383144280332632065',
 'user__id_str': '1187042015382507520'}

In [ ]:
tweets_constituyentes = twitter_db.tweets.find(
    { "user.id_str": {"$in": candidates_ids},
    "retweeted_status": { "$exists": False } }, no_cursor_timeout=True
)

buffer = []
for status in tqdm.tqdm(tweets_constituyentes, total=150000):
  if status["id_str"] in downloaded_ids:
    continue
  buffer.append(status)
  if len(buffer) >= 160:
    print(f"Processing {len(buffer)} tweets")
    tweets_text = [status["full_text"] for status in buffer]
    sentiment = analyzer.predict(tweets_text)
    out = [create_output_sentiment_analysis(i, sentiment, buffer) for i in range(len(buffer))]
    bulk_write_to_mongo(twitter_db.emotion_analysis, out)
    buffer = []


/usr/local/lib/python3.7/dist-packages/pymongo/collection.py:1532: UserWarning: use an explicit session with no_cursor_timeout=True otherwise the cursor may still timeout after 30 minutes, for more info see https://docs.mongodb.com/v4.4/reference/method/cursor.noCursorTimeout/#session-idle-timeout-overrides-nocursortimeout
  return Cursor(self, *args, **kwargs)
  0%|          | 102/150000 [00:01<28:03, 89.04it/s]

Processing 160 tweets


  0%|          | 0/5 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 160
  Batch size = 32


160


...

Processing 160 tweets


  0%|          | 0/5 [00:00<?, ?ba/s]

The following columns in the test set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Prediction *****
  Num examples = 160
  Batch size = 32


160


 99%|█████████▊| 147927/150000 [55:31<00:46, 44.41it/s]
